Final general cleanup of loans under 150k from all states and US territories 

In [219]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import os, sys 
import io
import ee


In [220]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [221]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

In [222]:
# !pip install -U -q PyDrive

In [223]:
#mounting the google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [224]:
# command shows file path
# ! ls

In [225]:
from google.colab import files

In [226]:
# filepath
path = "/content/drive/My Drive/data/quasi2.csv"




In [227]:
#read the csv, create dataframe
df = pd.read_csv(path,)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [228]:
df.head(2)

,Loan Amount,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank
0,149961.00,KANSAS CITY,MO,64108,541990.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,13,04/13/2020,Country Club Bank
1,149927.67,O FALLON,MO,63366,722511.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,NaN,04/07/2020,First State Bank of St. Charles


In [229]:
#shape
df.shape


(4224119, 13)

In [230]:
#check types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4224119 entries, 0 to 4224118
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Loan Amount    float64
 1   City           object 
 2   State          object 
 3   Zip            object 
 4   NAICS Code     float64
 5   Business Type  object 
 6   Race           object 
 7   Gender         object 
 8   Veteran        object 
 9   Non Profit     object 
 10  Jobs Saved     object 
 11  Date           object 
 12  Bank           object 
dtypes: float64(2), object(11)
memory usage: 419.0+ MB


In [231]:
print(df.isna().sum())

Loan Amount            0
City                 248
State                871
Zip                  208
NAICS Code        126705
Business Type       3330
Race                   3
Gender                 0
Veteran                0
Non Profit       4083421
Jobs Saved        285014
Date                  81
Bank                   0
dtype: int64


check jobs saved column


In [232]:
# PPP loans were only given to people who had "employees", however self employed people
#are counted as an employee for the purpose of the loan. Best guess is that recipients of loan
# below 25k would be fairly small business and will count as 1 employee. Not sure for values above that.
df.loc[df['Loan Amount'] <= 25000.00, "Jobs Saved"] = "1"
#check for nulls
print(df.isna().sum())

Loan Amount            0
City                 248
State                871
Zip                  208
NAICS Code        126705
Business Type       3330
Race                   3
Gender                 0
Veteran                0
Non Profit       4083421
Jobs Saved        113631
Date                  81
Bank                   0
dtype: int64


In [233]:
#dropping rows with zero value in column
df['Jobs Saved'].fillna(df['Jobs Saved'].mode().values[0], inplace = True)

In [234]:
print(df.isna().sum())

Loan Amount            0
City                 248
State                871
Zip                  208
NAICS Code        126705
Business Type       3330
Race                   3
Gender                 0
Veteran                0
Non Profit       4083421
Jobs Saved             0
Date                  81
Bank                   0
dtype: int64


check non profit coluumn

In [235]:
df['Non Profit'].value_counts()

Y              139179
Unanswered       1198
Non-Veteran       310
Veteran            11
Name: Non Profit, dtype: int64

In [236]:
np_df=df[df['Non Profit'] == 'Y']
np_df.sample()

,Loan Amount,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank
3379380,87630.0,CORNELIUS,NC,28031,519190.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,0.0,05/11/2020,Wells Fargo Bank


split non profit "Y" values off into separate csv to explore later as a table in sql and then drop the column. As a percentage of values, it seems like there isn't enough information as a percentage of the total

In [237]:
df.to_csv('non_profit.csv', index=0)
!cp non_profit.csv "drive/My Drive/data"

In [238]:
# drop non profit column. 
df = df.drop('Non Profit', 1)


Dropping null values from rows in several columns


In [239]:
#dropping null values from rows in the following columns
#another option wold be to fill jobs saved in with a value of one because it seems likely that 0 was likely from sole proprietors
df=df.dropna(subset=['Zip', 'Race', 'State', 'City', 'Date', 'Jobs Saved', 'NAICS Code', 'Business Type'])
print(df.isna().sum())



Loan Amount      0
City             0
State            0
Zip              0
NAICS Code       0
Business Type    0
Race             0
Gender           0
Veteran          0
Jobs Saved       0
Date             0
Bank             0
dtype: int64


In [240]:
#percentage of rows lost. 3% - not bad
dif=4224119-4093662
print(dif/4224119)

0.030883836369193198


In [241]:
#lost about 3% of rows
df.shape

(4093662, 12)

NAICS Codes

In [242]:
df['NAICS Code'].value_counts()

722511.0    136768
531210.0    111306
541110.0    108156
621210.0     89360
621111.0     88102
             ...  
48009.0          1
48034.0          1
48043.0          1
48073.0          1
623.0            1
Name: NAICS Code, Length: 1745, dtype: int64

In [243]:
#naics saved should not be a float; convert to string before integer
df['NAICS Code']=df['NAICS Code'].astype(str)

In [244]:
#strip the decimal and zero
df['NAICS Code'] = df['NAICS Code'].str.rstrip('.0')

In [245]:
#take the first two letters of the NAICS code column and create category column. 
#category column can be matched to named values. 
df['NAICS Category'] = df['NAICS Code'].str[:2]
df['NAICS Category'].dtype

dtype('O')

In [246]:
df.sample()

,Loan Amount,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Jobs Saved,Date,Bank,NAICS Category
2868149,62300.0,NORTH RICHLAND HILLS,TX,76182,81311,Non-Profit Organization,Unanswered,Male Owned,Unanswered,0.0,04/28/2020,The American National Bank of Texas,81


In [247]:
df.shape

(4093662, 13)

Date column

In [248]:
# somewhat forcing the date conversion because there must be some values that are not able to be converted to date format 
df['Date'] =  pd.to_datetime(df['Date'], errors='coerce')

In [249]:
df.shape

(4093662, 13)

In [250]:
#check for stray values
# df.Date.value_counts(ascending=True)

In [251]:
# date conversion verification 
df['Date'].dtype

dtype('<M8[ns]')

In [252]:
#check 
df.sample(3)

,Loan Amount,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Jobs Saved,Date,Bank,NAICS Category
3543699,26245.0,NASHVILLE,TN,37215,561311,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,4,2020-05-03,Bank of America,56
3320841,15010.0,PRINGLE,PA,18704,56172,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,1,2020-05-03,Celtic Bank Corporation,56
1707773,10200.0,MESA,AZ,85201,812113,Sole Proprietorship,Unanswered,Unanswered,Unanswered,1,2020-05-07,MidFirst Bank,81


State column


In [253]:
#number of unique values in a column
df['State'].nunique()

273

In [254]:
df['State'].value_counts()

CA            470710
FL            341051
TX            328235
NY            261809
IL            167429
               ...  
 NY 11206"         1
 AR 72405"         1
FI                 1
 SD"               1
 OR 97045"         1
Name: State, Length: 273, dtype: int64

In [255]:
#remove open space to the left side of state
df['State'] = df['State'].str.strip(' ', )

In [256]:
df['State'].value_counts()

CA            470710
FL            341051
TX            328235
NY            261809
IL            167429
               ...  
FL  33931"         1
OR 97045"          1
STE. 513"          1
WA 98001"          1
NJ 07306"          1
Name: State, Length: 260, dtype: int64

In [216]:
df['State'] = df.loc[df['State']].str.rstrip('"', )
#remove the stray "

In [265]:
#taking first two letters 
df['State'] = [x[:2] for x in df['State']]


In [278]:
df=df.groupby('State').filter(lambda x : (x['State'].count()>=3).any())

In [279]:
df['State'].value_counts()

CA    470781
FL    341087
TX    328286
NY    261850
IL    167444
GA    136190
PA    136094
NJ    116632
OH    112418
NC    103574
MI     97110
MA     92441
VA     91757
CO     88693
MN     83186
WA     82909
TN     80744
MO     78890
WI     71637
AZ     67140
MD     67078
IN     65879
LA     63298
OK     57110
AL     57072
SC     54081
IA     52229
OR     52056
CT     50809
KS     45585
UT     42622
MS     41673
KY     41130
NE     38139
AR     37931
NV     33882
PR     31622
ID     26316
ME     23993
HI     21189
MT     20768
SD     20490
NH     19682
NM     18579
ND     17491
WV     14895
RI     14467
WY     11650
VT     10229
DE      9991
AK      9412
DC      9199
GU      1867
VI      1686
MP       417
AS       204
XX        16
TE        11
ST         7
Name: State, dtype: int64

In [281]:
df.shape

(4093618, 13)

In [182]:
# df['State'] = df['State'].str.strip('.', )

In [ ]:
# df.shape

In [ ]:
# df['States'] = df['States'].str.lstrip(' ', )

In [282]:
df['State'].nunique()

59

In [284]:
df['NAICS Category'].value_counts()

54    553820
81    491278
62    422097
23    380205
72    308883
44    261611
53    227682
56    208301
48    159261
52    154859
45    137492
42    129498
11    129384
71    108352
33     86468
99     83900
61     67447
51     57559
32     38651
31     35583
21     16388
92     11333
49      9824
55      7236
22      6137
78        28
10        22
20        17
77        14
91        14
85        14
60        13
80        13
94        12
95        11
90        10
68        10
75         9
76         9
30         9
82         8
98         8
96         8
93         8
70         8
28         7
73         7
14         6
89         6
26         5
34         5
74         5
13         5
64         5
97         5
24         5
12         4
27         4
88         4
19         4
79         4
17         4
18         4
37         4
58         3
84         3
66         3
63         2
35         2
57         2
50         2
29         2
86         2
36         1
39         1
69         1
59         1

In [285]:
df = df.groupby('NAICS Category').filter(lambda x: len(x) >= 5)

In [287]:
# cleaned file of loans under $150k preserving roughly 4 million rows
df.to_csv('jobs_saved.csv', index=0)
!cp jobs_saved.csv "drive/My Drive/"

In [ ]:
col=list(df.columns)

In [ ]:
#convert the strings to integers in jobs saved
# df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], errors="coerce")
# df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
# df['Jobs Saved'].fillna(0, inplace = True)
df.shape

In [ ]:
df.dtypes

In [ ]:
df.sample(12)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# df.to_csv('clean_ppp.csv', index=0)
# !cp clean_ppp.csv "drive/My Drive/"


In [ ]:
# data looks pretty good. 
df.head()